In [1]:
import numpy as np
import matplotlib.pyplot as plt
import healpy as hp
import subprocess
from lsst.sims.featureScheduler.features import N_observations
from lsst.sims.featureScheduler.utils import hp_in_lsst_fov, schema_converter
import os
from lsst.sims.utils import Site
import shutil
import sys
import sqlite3 as db
import pandas as pd


In [2]:
minion = False
if minion:
    con = db.connect('minion_1016_sqlite.db')
    df = pd.read_sql('select fieldRA,fieldDec,lst,filter,night from Summary;', con)
    names = ['RA', 'dec', 'lmst','filter','night']
    types = [float, float, float, '|U1',int]
    observations = np.empty(df.shape[0], dtype=list(zip(names,types)))
    observations['RA'] = df['fieldRA'].values
    observations['dec'] = df['fieldDec'].values
    observations['lmst'] = df['lst'].values*12/np.pi
    observations['filter'] = df['filter'].values
    observations['night'] = df['night'].values
    moviename = 'minion_movie.mp4'
else:
    # Load up observations
    dbfile = 'baseline_1exp_pairsmix_10yrs.db'
    sc = schema_converter()
    observations = sc.opsim2obs(dbfile)
    moviename = 'baseline_movie.mp4'


  


In [3]:
night = 23
outdir = 'temp'
if os.path.isdir(outdir):
    shutil.rmtree(outdir)
os.mkdir(outdir)
nside = 128
if os.path.isfile(moviename):
    os.remove(moviename)


site = Site('LSST')

In [4]:
# make a feature
feature = N_observations(nside=nside)
hplookup = hp_in_lsst_fov(nside=nside)

empty = feature.feature + hp.UNSEEN
good = np.where(observations['night'] == night)[0]

In [5]:
max_i = np.size(good)
for i,observation in enumerate(observations[good]):
    indx = hplookup(observation['RA'], observation['dec'])
    feature.add_observation(observation, indx=indx)
    # Could insert an if here to only output a frame every N observations
    good_pix = np.where(feature.feature > 0)
    empty[good_pix] = feature.feature[good_pix]
    rot = (observation['lmst']*360./24, 0) #site.latitude)
    hp.mollview(empty, min=0, max=10, title='', rot=rot)
    plt.savefig(os.path.join(outdir, '%i.png' % i))
    plt.close('all')
    progress = i/float(max_i)*100
    text = "\rprogress = %.1f%%" %progress
    sys.stdout.write(text)
    sys.stdout.flush()

progress = 99.9%

In [6]:
subprocess.call("ffmpeg -i %s/%%d.png  -f mp4 -framerate 10 -pix_fmt yuv420p  %s" % (outdir, moviename), shell=True)

0